In [1]:
from datetime import datetime
import os

import pandas as pd
from sqlalchemy import exc, create_engine

In [2]:
def sql_connection(rds_schema: str):
    """
    SQL Connection function connecting to my postgres db with schema = nba_source where initial data in ELT lands.
    Args:
        rds_schema (str): The Schema in the DB to connect to.
    Returns:
        SQL Connection variable to a specified schema in my PostgreSQL DB
    """
    RDS_USER = os.environ.get("RDS_USER")
    RDS_PW = os.environ.get("RDS_PW")
    RDS_IP = os.environ.get("IP")
    RDS_DB = os.environ.get("RDS_DB")
    try:
        connection = create_engine(
            f"postgresql+psycopg2://{RDS_USER}:{RDS_PW}@{RDS_IP}:5432/{RDS_DB}",
            connect_args={"options": f"-csearch_path={rds_schema}"},
            # defining schema to connect to
            echo=False,
        )
        return connection
    except exc.SQLAlchemyError as e:
        return e

def write_to_sql(con, table_name: str, df: pd.DataFrame, table_type: str):
    """
    SQL Table function to write a pandas data frame to SQL
    Args:
        con (SQL Connection): The connection to the SQL DB.
        table_name (str): The Table name to write to SQL as.
        df (DataFrame): The Pandas DataFrame to store in SQL
        table_type (str): Whether the table should replace or append to an existing SQL Table under that name
    Returns:
        Writes the Pandas DataFrame to a Table in SQL
    """
    try:
        if len(df) == 0:
            print(f"{table_name} is empty, not writing to SQL")
        else:
            df.to_sql(
                con=con,
                name=f"{table_name}",
                index=False,
                if_exists=table_type,
            )
    except BaseException as error:
        return error

conn = sql_connection(rds_schema='fanduel')

In [3]:
customers = pd.read_excel('fanduel_data.xlsx', sheet_name = 'customers')
sportsbook_bets = pd.read_excel('fanduel_data.xlsx', sheet_name = 'sportsbook_bets')
casino = pd.read_excel('fanduel_data.xlsx', sheet_name = 'casino')
sportsbook_totals = pd.read_excel('fanduel_data.xlsx', sheet_name = 'sportsbook_totals')

In [6]:
write_to_sql(conn, 'customers', customers, 'replace')
write_to_sql(conn, 'sportsbook_bets', sportsbook_bets, 'replace')
write_to_sql(conn, 'casino', casino, 'replace')
write_to_sql(conn, 'sportsbook_totals', sportsbook_totals, 'replace')